In [ ]:
from pathlib import Path
import subprocess


def run_dcm2nii_over_bids(
    bids_root: str | Path,
    dcm2nii_cmd: str = "dcm2niix",
    sub_folder: str = "sub-",
    ses_folder: str = "ses-",
    dcm_folder: str = "DCM",
    fname: str = "T1"
) -> None:
    """
    Convert every /DCM folder in a BIDS tree to NIfTI and drop
    the output in the matching /ANAT folder.

    Layout expected
    ---------------
    <bids_root>/
        sub-XXX/
            ses-YYY/
                DCM/      <-- raw DICOM files (already organised)
                ANAT/     <-- will receive the NIfTI output
    """
    bids_root = Path(bids_root).expanduser().resolve()

    # locate every DCM directory: sub-*/ses-*/DCM
    for dcm_dir in bids_root.glob(f"{sub_folder}*/{ses_folder}*/{dcm_folder}"):
        if not dcm_dir.is_dir():
            continue

        anat_dir = dcm_dir.parent / "ANAT"
        anat_dir.mkdir(exist_ok=True)

        try:
            subprocess.run(
                [dcm2nii_cmd, "-b", "y", "-z", "y", "-o", anat_dir, str(dcm_dir)],
                check=True,
            )
        except subprocess.CalledProcessError as e:
            print(f"Error processing {dcm_dir}: {e}")
            continue

        # OPTIONAL — rename first *.nii* to sub-XXX_T1.nii.gz
        nifti_files = sorted(anat_dir.glob("*.nii*"))
        if len(nifti_files) == 1:
            subj_id = dcm_dir.parents[1].name.lstrip("sub-")
            nifti_files[0].rename(anat_dir / f"{fname}.nii.gz")

In [2]:
run_dcm2nii_over_bids('/Users/cu135/Partners HealthCare Dropbox/Calvin Howard/studies/seeg_memory/patients', 'dcm2niix',
                      sub_folder = 'sub-14743546')

Chris Rorden's dcm2niiX version v1.0.20230411  Clang14.0.3 ARM (64-bit MacOS)
Found 1 DICOM file(s)
Convert 1 DICOM as /Users/cu135/Partners HealthCare Dropbox/Calvin Howard/studies/seeg_memory/patients/sub-14743546/ses-01/ANAT/DCM_Pre_Ax_3D_MPRAGE_T1_20250909114712_8 (216x256x192x1)
Conversion required 2.319330 seconds (2.290999 for core code).
